In [209]:
import os
from ast import literal_eval
import pandas as pd
from pathlib import Path
from environ import Env
import pickle
import warnings
warnings.filterwarnings('ignore')

# Build paths inside the project like this: BASE_DIR / 'subdir'.
BASE_DIR = Path(os.curdir).resolve().parent.parent
env = Env()
env_path = BASE_DIR / "django/.env"
if env_path.exists():
    with env_path.open("rt", encoding="utf8") as f:
        env.read_env(f, overwrite=True)

dbname = env.get_value('GCPDB_NAME')
user = env.get_value('GCPDB_USER')
pw = env.get_value('GCPDB_PASSWORD')
host = env.get_value('GCPDB_HOST')

from sqlalchemy import create_engine
# echo를 true로 설정하면 command창에 실행된 sql문이 뜨게 됨
engine = create_engine(f'mysql+mysqldb://{user}:{pw}@{host}:3306/{dbname}?charset=utf8')

PKL_DIR = Path(os.curdir).resolve().parent.parent
PKL_DIR = PKL_DIR / "Utils/Pickle"

In [157]:
!ls ../../Data/EDA/ml-25m/

README.txt	   genome-tags.csv  movies.csv	 tags.csv
genome-scores.csv  links.csv	    ratings.csv


In [158]:
!ls /opt/ml/project2/Utils/Pickle/23*

/opt/ml/project2/Utils/Pickle/230130_Popular_movie_1192_cwj.pickle
/opt/ml/project2/Utils/Pickle/230130_Popular_movie_character_2867_cwj.pickle
/opt/ml/project2/Utils/Pickle/230131_watch_link_4679_rows.pickle
/opt/ml/project2/Utils/Pickle/230201_characterid_to_hashtag.pickle
/opt/ml/project2/Utils/Pickle/230201_fit_mbti_dict.pickle
/opt/ml/project2/Utils/Pickle/230203_character_movie_merge.pickle


In [159]:
rating = pd.read_csv('/opt/ml/project2/Data/EDA/ml-25m/ratings.csv')

In [160]:
cdf = pd.read_pickle(PKL_DIR / '230130_Popular_movie_character_2867_cwj.pickle')
mdf = pd.read_pickle(PKL_DIR / '230130_Popular_movie_1192_cwj.pickle')

## Fake User DB에서 불러오기

In [161]:
tmp_user = pd.read_sql_query("select * from test_rec_tmpuser", engine)
auth_user = pd.read_sql_query("select * from auth_user", engine)

In [162]:
auth_cols=['id','username']
tmp_cols = ['LoginUser_id','MBTI','ennea_res','prefer_movie_id','create_time']

In [164]:
real_user = auth_user[auth_user.id.notna()]
real_user

,id,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
0,1,pbkdf2_sha256$390000$R5F879x1Qstp2V6H3n82xy$xY...,2023-02-04 14:42:41.527730,1,cwj,,,cwj@naver.com,1,1,2023-01-29 13:01:28.048388
1,2,!mmQE7zcvRQ9pcGnOtyEMySzGF8peQu62SKVirA6o,2023-02-03 20:43:39.013448,0,awsed2486,나현,이,awsed2486@gmail.com,0,1,2023-01-29 18:14:11.020115
2,3,!aI602NFLAewkSwRp5QP5T7lQQWbF7MGlxmCtylBC,2023-02-04 19:07:29.401230,0,cwjcsk,해피,조,cwjcsk@gmail.com,0,1,2023-01-30 14:12:47.146695
3,4,pbkdf2_sha256$390000$QnKYRE2d9GtfhDwppjXsOG$NQ...,2023-01-30 16:53:31.294270,0,이나현,,,abc@example.com,0,1,2023-01-30 16:53:30.689159
4,5,!z9ClJWiGykoETbT2ix1b34UbXAVU0cgDP6XhnKi4,2023-02-02 06:35:30.766830,0,uiuh4569,의준,정,uiuh4569@gmail.com,0,1,2023-01-30 17:26:31.180120
5,6,!jIAXteJtY9rCgJdyDxU17H7wLh3AUFzgHqe2Ot3W,2023-02-04 07:16:07.978224,0,jhealee0624,해리,전,jhealee0624@gmail.com,0,1,2023-02-01 02:25:04.124092
6,7,!mFAzZvAUrUmOQ2rXFFALIXnTNNmaspOo60Kx69Ym,2023-02-03 12:33:10.144832,0,garam,GARAM,LEE,garamlee13@gmail.com,0,1,2023-02-01 03:55:33.083219
7,8,!6og98zNMBLrMgGkzj9MwEJmKgmIwBDbBj4L0jUXf,2023-02-04 15:48:58.223036,0,larila,larila,bam,guswn2521@gmail.com,0,1,2023-02-01 06:09:57.037632
8,9,pbkdf2_sha256$390000$DeJuuplcPgybmGLMR3lnt9$by...,2023-02-04 19:54:29.902005,0,nahyun2,,,asdf@asdf.com,0,1,2023-02-01 20:09:24.296057
9,10,!kJ1PRib2y6qIyZkyfDBg0mVTitx2yQ1pLiCyS5NH,2023-02-02 14:37:24.054752,0,juntaegim99,준태,김,juntaegim99@gmail.com,0,1,2023-02-02 14:37:23.989346


In [165]:
real_user_df = real_user[auth_cols].merge(tmp_user[tmp_cols], left_on='id',right_on='LoginUser_id')
real_user_df.prefer_movie_id = real_user_df.prefer_movie_id.map(literal_eval)
real_user_df

,id,username,LoginUser_id,MBTI,ennea_res,prefer_movie_id,create_time
0,1,cwj,1.0,INTJ,1w9,"[45672, 68135, 86833, 6957, 56941, 111622, 341...",2023-01-30 17:44:27.678267
1,1,cwj,1.0,ISFJ,1w2,"[7346, 97938, 105197, 3752, 2336]",2023-01-31 03:32:02.853235
2,1,cwj,1.0,INTJ,7w6,[],2023-01-31 04:49:27.604951
3,1,cwj,1.0,INFJ,5w4,"[300033, 2587, 166946, 300075, 124476]",2023-02-01 06:24:08.584533
4,1,cwj,1.0,INTJ,1w9,"[117881, 97921, 166534, 73881, 81562, 101112, ...",2023-02-01 12:16:41.525240
...,...,...,...,...,...,...,...
85,11,minsu,11.0,ENTP,2w3,"[111622, 208939, 164909, 48, 588]",2023-02-04 06:21:13.762982
86,13,nahyun3,13.0,INTP,1w9,"[300033, 81932, 69134, 97304, 182811, 64034, 7...",2023-02-03 20:46:04.098496
87,13,nahyun3,13.0,INFJ,1w9,"[91653, 6662, 74255, 300054, 97304, 541, 1566]",2023-02-03 20:52:50.042156
88,13,nahyun3,13.0,ENTP,4w3,"[157699, 69640, 72720, 72209, 197199, 300116, ...",2023-02-03 21:04:23.108712


### MBTI, ennea_res가 바뀐 다면 create_time이 가장 마지막인 것을 기준으로 추천하기

## 기존 mdf와 MovieLens로 Interaction 만들기

In [166]:
mdf_rating = rating[rating.movieId.isin(mdf.movieId)]
mdf_rating

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
9,1,1250,4.0,1147868414
11,1,1653,4.0,1147868097
23,1,3949,5.0,1147868678
25,1,4308,3.0,1147868534
...,...,...,...,...
25000081,162541,7361,4.5,1240953484
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697


In [167]:
mdf_rating.rating=1
mdf_rating[:2]

,userId,movieId,rating,timestamp
0,1,296,1,1147880044
9,1,1250,1,1147868414


## Fake User의 Interaction 합치기

### Fake User의 userID 변환 (Django DB user id: rating에 쓰일 user id)

In [168]:
mdf_rating.userId.max()

162541

In [169]:
common_cols = ['userId','movieId']

In [202]:
real_user_inter = real_user_df[['id','prefer_movie_id']].explode('prefer_movie_id')
# real_user_inter['rating']=1
real_user_inter.columns=common_cols
print(f"유저별 중복 영화 제거 전: {real_user_inter.shape[0]}")
real_user_inter.dropna(inplace=True)
real_user_inter.drop_duplicates(subset=['userId','movieId'], inplace=True)
print(f"유저별 중복 영화 제거 후: {real_user_inter.shape[0]}")
real_user_inter.userId+=300000
real_user_inter.movieId = real_user_inter.movieId.astype(int)
real_user_inter[:10]

유저별 중복 영화 제거 전: 543
유저별 중복 영화 제거 후: 438


,userId,movieId
0,300001,45672
0,300001,68135
0,300001,86833
0,300001,6957
0,300001,56941
0,300001,111622
0,300001,34162
0,300001,106782
0,300001,8644
0,300001,189203


### 합치기

In [203]:
mdf_rating.shape, real_user_inter.shape

((6348460, 4), (438, 2))

In [204]:
merge_rating = pd.concat([mdf_rating[common_cols],real_user_inter])
merge_rating.shape

(6348898, 2)

## Recbole input 만들기 (.inter, .data 등)

In [114]:
!ls ../../BaseLine/dataset/train_data/
!readlink -ef ../../BaseLine/dataset/train_data/

train_data.inter
/opt/ml/project2/BaseLine/dataset/train_data


In [205]:
merge_rating.columns = ['user_id:token', 'item_id:token']
merge_rating[:3]

,user_id:token,item_id:token
0,1,296
9,1,1250
11,1,1653


In [182]:
merge_rating.to_csv("/opt/ml/project2/BaseLine/dataset/train_data/train_data.inter",sep='\t',index=False)

In [188]:
userid2idx = {v:k for k,v in enumerate(sorted(set(merge_rating['user_id:token'])))}

In [207]:
itemid2idx = {v:k for k,v in enumerate(sorted(set(merge_rating['item_id:token'])))}

## EASE 학습 및 추천

In [210]:
df = pd.read_sql_query("select * from common_batchtrain", engine)
df.head()

,id,LoginUser_id,model_path,recommended_movie_list,create_time


In [154]:
cols = ['id','LoginUser_id', 'model', 'recommended_movie_list','create_time']
df = pd.DataFrame(columns=cols)
df.loc[len(df),:]=[len(df),3,"EASE2","[1,2,3]",pd.Timestamp.now()]
df

,id,LoginUser_id,model,recommended_movie_list,create_time
0,0,3,EASE2,"[1,2,3]",2023-02-05 04:50:33.392360


In [155]:
df.to_sql(name='common_batchtrain', con=engine, index=False, if_exists='append')

1

In [ ]:
LoginUser_id=3
model="EASE"
recommended_movie_list=[1,2,3]

In [ ]:
## 추천 movieids 추출

In [ ]:
## DB에 저장

In [117]:
fake_user_inter.to_sql('table_name',engine)

41

In [ ]:
## TmpUser와 User의 관계가 어떻게 되는지, 어떻게 추천을 해줄 수 있는지 내일 나현님께 여쭤보기
# -> 답변완료